In [6]:
import os
import shutil
import random
from PIL import Image
import numpy as np
from cleanfid import fid
import cv2
from os import listdir
from os.path import isfile, join
from tqdm import tqdm
import torch
import torch.nn.functional as F
from torchvision import transforms
from glob import glob
from itertools import combinations
import pandas as pd

In [7]:
# Set random seed for reproducibility
random.seed(73)
np.random.seed(73)
torch.manual_seed(73)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
# Utility functions
def pil_to_tensor(image):
    return transforms.ToTensor()(image).unsqueeze(0).to(device)

def tensor_to_pil(tensor):
    return transforms.ToPILImage()(tensor.squeeze(0).cpu())

In [9]:
def downscale_upscale(image, scale_factor=0.3):
    """
    Downscale and then upscale the image to remove fine details using PyTorch.
    
    Args:
        image (torch.Tensor): Input image tensor (1, C, H, W).
        scale_factor (float): Scaling factor for downscaling.

    Returns:
        torch.Tensor: Image with reduced details.
    """
    # Ensure input is 4D tensor (batch, channel, height, width)
    if len(image.shape) == 3:
        image = image.unsqueeze(0)  # Add batch dimension if missing
    
    _, c, h, w = image.shape
    
    # Calculate new dimensions (minimum size of 1 to prevent errors)
    new_h = max(1, int(h * scale_factor))
    new_w = max(1, int(w * scale_factor))
    
    # Downscale
    downscaled = F.interpolate(image, size=(new_h, new_w), mode='bilinear', align_corners=False)
    
    # Upscale back to original size
    upscaled = F.interpolate(downscaled, size=(h, w), mode='bicubic', align_corners=False)
    
    return upscaled

In [10]:
def apply_gaussian_blur(image, kernel_size=5, sigma=1.0):
    """
    Apply Gaussian blur to the image using PyTorch.
    
    Args:
        image (torch.Tensor): Input image tensor (1, C, H, W).
        kernel_size (int): Size of the Gaussian kernel.
        sigma (float): Standard deviation of the Gaussian kernel.

    Returns:
        torch.Tensor: Blurred image.
    """
    # Create 1D Gaussian kernel
    x = torch.arange(kernel_size, dtype=torch.float32, device=device) - kernel_size // 2
    kernel_1d = torch.exp(-x.pow(2) / (2 * sigma ** 2))
    kernel_1d = kernel_1d / kernel_1d.sum()
    
    # Create 2D kernel
    kernel_2d = torch.outer(kernel_1d, kernel_1d).unsqueeze(0).unsqueeze(0)
    
    # Apply padding
    padding = kernel_size // 2
    
    # Apply convolution to each channel separately
    channels = []
    for c in range(image.shape[1]):
        channel = image[:, c:c+1, :, :]
        blurred = F.conv2d(channel, kernel_2d, padding=padding)
        channels.append(blurred)
    
    return torch.cat(channels, dim=1)

In [11]:
def apply_gaussian_blur(image, kernel_size=5, sigma=1.0):
    """
    Apply Gaussian blur to the image using PyTorch.
    
    Args:
        image (torch.Tensor): Input image tensor (1, C, H, W).
        kernel_size (int): Size of the Gaussian kernel.
        sigma (float): Standard deviation of the Gaussian kernel.

    Returns:
        torch.Tensor: Blurred image.
    """
    # Create 1D Gaussian kernel
    x = torch.arange(kernel_size, dtype=torch.float32, device=device) - kernel_size // 2
    kernel_1d = torch.exp(-x.pow(2) / (2 * sigma ** 2))
    kernel_1d = kernel_1d / kernel_1d.sum()
    
    # Create 2D kernel
    kernel_2d = torch.outer(kernel_1d, kernel_1d).unsqueeze(0).unsqueeze(0)
    
    # Apply padding
    padding = kernel_size // 2
    
    # Apply convolution to each channel separately
    channels = []
    for c in range(image.shape[1]):
        channel = image[:, c:c+1, :, :]
        blurred = F.conv2d(channel, kernel_2d, padding=padding)
        channels.append(blurred)
    
    return torch.cat(channels, dim=1)

In [12]:
def apply_median_filter(image, kernel_size=5):
    """
    Apply a median filter to the image using PyTorch.
    
    Args:
        image (torch.Tensor): Input image tensor (1, C, H, W).
        kernel_size (int): Size of the median filter kernel.

    Returns:
        torch.Tensor: Filtered image.
    """
    padding = kernel_size // 2
    padded = F.pad(image, (padding, padding, padding, padding), mode='reflect')

    # Extract shape
    b, c, h, w = image.shape
    unfolded = F.unfold(padded, kernel_size=kernel_size)

    # Now reshape using actual dimensions
    unfolded = unfolded.view(b, c, kernel_size * kernel_size, h * w)
    median = torch.median(unfolded, dim=2)[0]

    return median.view(b, c, h, w)

In [13]:
def apply_bilateral_filter(image, sigma_space=5, sigma_color=0.1, kernel_size=5):
    """
    Fixed bilateral filtering using PyTorch.
    Args:
        image (torch.Tensor): Input image tensor (1, C, H, W).
        sigma_space (float): Spatial standard deviation.
        sigma_color (float): Color standard deviation.
        kernel_size (int): Size of the filter kernel.
    Returns:
        torch.Tensor: Filtered image.
    """
    padding = kernel_size // 2
    b, c, h, w = image.shape
    
    # Create spatial kernel
    grid = torch.stack(torch.meshgrid(
        torch.arange(kernel_size, device=device), 
        torch.arange(kernel_size, device=device), indexing='ij'))
    center = kernel_size // 2
    spatial_dist = ((grid - center) ** 2).sum(dim=0).sqrt()
    spatial_kernel = torch.exp(-0.5 * (spatial_dist / sigma_space) ** 2)
    spatial_kernel = spatial_kernel.view(1, -1, 1)
    
    # Unfold the image
    unfolded = F.unfold(image, kernel_size=kernel_size, padding=padding)
    unfolded = unfolded.view(b, c, kernel_size*kernel_size, h*w)
    
    # Compute center pixels (using mean of neighborhood)
    center_pixels = F.avg_pool2d(image, kernel_size=kernel_size, stride=1, padding=padding)
    center_pixels = center_pixels.view(b, c, 1, h*w)
    
    # Compute color distances
    color_dist = (unfolded - center_pixels).abs().mean(dim=1)  # Mean across channels
    color_kernel = torch.exp(-0.5 * (color_dist / sigma_color) ** 2)
    
    # Combine kernels
    combined_kernel = spatial_kernel * color_kernel
    combined_kernel = combined_kernel / (combined_kernel.sum(dim=1, keepdim=True) + 1e-8)
    
    # Apply filter
    filtered = (unfolded * combined_kernel.unsqueeze(1)).sum(dim=2)
    return filtered.view(b, c, h, w)

In [14]:
# @torch.compile(mode="reduce-overhead")
def apply_color_quantization(image, n_colors=64):
    """
    Fixed color quantization implementation.
    Args:
        image (torch.Tensor): Input image tensor (1, C, H, W).
        n_colors (int): Number of colors to quantize to.
    Returns:
        torch.Tensor: Quantized image.
    """
    # Reshape to pixels x channels
    b, c, h, w = image.shape
    pixels = image.permute(0, 2, 3, 1).reshape(-1, c)
    
    # Simple k-means clustering (approximate)
    k = min(n_colors, len(pixels))
    centroids = pixels[torch.randperm(len(pixels))[:k]]
    
    for _ in range(5):  # Reduced iterations for speed
        distances = torch.cdist(pixels, centroids)
        labels = distances.argmin(dim=1)
        for i in range(k):
            if (labels == i).any():
                centroids[i] = pixels[labels == i].mean(dim=0)
    
    # Replace pixels with nearest centroid
    distances = torch.cdist(pixels, centroids)
    quantized_pixels = centroids[distances.argmin(dim=1)]
    
    return quantized_pixels.view(b, h, w, c).permute(0, 3, 1, 2)

In [15]:
# @torch.compile()
def enhance_edges(image, edge_threshold=0.2, enhancement_factor=2.0):
    sobel_x = torch.tensor([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=torch.float32, device=device).view(1, 1, 3, 3)
    sobel_y = torch.tensor([[-1, -2, -1], [0, 0, 0], [1, 2, 1]], dtype=torch.float32, device=device).view(1, 1, 3, 3) 
    grad_x = F.conv2d(image.mean(dim=1, keepdim=True), sobel_x, padding=1)
    grad_y = F.conv2d(image.mean(dim=1, keepdim=True), sobel_y, padding=1)
    edge_magnitude = torch.sqrt(grad_x**2 + grad_y**2)
    edge_mask = (edge_magnitude > edge_threshold).float()
    enhanced = image * (1 + (enhancement_factor - 1) * edge_mask)
    return torch.clamp(enhanced, 0, 1)

In [16]:
# @torch.compile()  
def apply_motion_blur(img_tensor, intensity):
    """
    Apply motion blur to an image tensor.
    
    Args:
        img_tensor (torch.Tensor): Input image tensor (CxHxW or BxCxHxW)
        intensity (float): Blur intensity (controls kernel size)
        
    Returns:
        torch.Tensor: Blurred image tensor
    """
    # Ensure we have batch dimension
    has_batch = len(img_tensor.shape) == 4
    if not has_batch:
        img_tensor = img_tensor.unsqueeze(0)
    
    # Get device from input tensor
    device = img_tensor.device
    
    # Create motion blur kernel
    kernel_size = max(3, int(intensity * 10))
    kernel = torch.zeros((kernel_size, kernel_size), device=device)
    kernel[kernel_size//2, :] = 1.0 / kernel_size
    
    # Expand for RGB channels
    kernel = kernel.unsqueeze(0).unsqueeze(0)  # (1,1,k,k)
    kernel = kernel.expand(3, -1, -1, -1)      # (3,1,k,k)
    
    # Apply convolution
    padding = kernel_size // 2
    result = torch.nn.functional.conv2d(img_tensor, kernel, padding=padding, groups=3)
    
    return result if has_batch else result.squeeze(0)

In [17]:
# @torch.compile()
def add_chromatic_aberration(img_tensor, max_shift):
    """
    Add chromatic aberration effect by shifting color channels.
    
    Args:
        img_tensor (torch.Tensor): Input image tensor (CxHxW or BxCxHxW)
        max_shift (float): Maximum pixel shift amount
        
    Returns:
        torch.Tensor: Image with chromatic aberration
    """
    # Ensure we have batch dimension
    has_batch = len(img_tensor.shape) == 4
    if not has_batch:
        img_tensor = img_tensor.unsqueeze(0)
    
    # Get spatial dimensions
    _, c, h, w = img_tensor.shape
    
    # Create shifted versions of each channel
    shifted = []
    for i in range(c):
        # Random shift in x and y directions
        shifts = torch.randint(-int(max_shift), int(max_shift)+1, (2,), device=img_tensor.device)
        shift_x, shift_y = shifts[0], shifts[1]
        
        # Apply shift
        channel = img_tensor[:, i:i+1]
        channel = torch.roll(channel, shifts=(shift_x, shift_y), dims=(-1, -2))
        shifted.append(channel)
    
    result = torch.cat(shifted, dim=1)
    return result if has_batch else result.squeeze(0)

In [18]:
DEGRADATION_FUNCTIONS = [
    {
        'name': 'downscale_upscale',
        'func': downscale_upscale,
        'params': {'scale_factor': (0.4, 0.7)}
    },
    {
        'name': 'enhance_edges',
        'func': enhance_edges,
        'params': {
            'edge_threshold': (0.15, 0.25),
            'enhancement_factor': (1.0, 1.3)
        }
    },
    {
        'name': 'gaussian_blur',
        'func': apply_gaussian_blur,
        'params': {
            'kernel_size': [3, 5, 7, 9],
            'sigma': (0.5, 1.5)
        }
    },
    {
        'name': 'median_filter',
        'func': apply_median_filter,
        'params': {
            'kernel_size': [3, 5, 7, 9]
        }
    },
    {
        'name': 'bilateral_filter',
        'func': apply_bilateral_filter,
        'params': {
            'sigma_space': (3, 5),
            'sigma_color': (0.05, 0.3),
            'kernel_size': [3, 5, 7, 9]
        }
    },
    {
        'name': 'color_quantization',
        'func': apply_color_quantization,
        'params': {
            'n_colors': [32, 64]
        }
    },
    {
        'name': 'motion_blur',
        'func': apply_motion_blur,
        'params': {
            'intensity': (0.2, 0.3)
        }
    },
    {
        'name': 'chromatic_aberration',
        'func': add_chromatic_aberration,
        'params': {
            'max_shift': (1.0, 2.0)
        }
    }
]

In [40]:
def generate_pipeline_combination(combination):
    """Create a pipeline function that only applies the specified combination of degradations"""
    def combo_pipeline(image):
        try:
            img_tensor = pil_to_tensor(image)
            
            if img_tensor.shape[1] == 4:
                img_tensor = img_tensor[:, :3, :, :]
            
            for func_info in combination:
                func = func_info['func']
                params = {}
                
                # Generate random parameters within specified ranges
                for param_name, param_range in func_info['params'].items():
                    if isinstance(param_range, (list, tuple)):
                        if isinstance(param_range[0], int):  # Discrete choices
                            params[param_name] = random.choice(param_range)
                        else:  # Continuous range
                            params[param_name] = np.random.uniform(param_range[0], param_range[1])
                
                try:
                    img_tensor = func(img_tensor, **params)
                except Exception as e:
                    print(f"{func_info['name']} failed: {e}")
                    continue
            
            return tensor_to_pil(img_tensor.clamp(0, 1))
        
        except Exception as e:
            print(f"Error processing image: {e}")
            return image
    
    return combo_pipeline

In [19]:
# Randomly select N images from a folder
def select_random_images(folder, n=1000):
    all_images = sorted(os.listdir(folder))
    return random.sample(all_images, min(n, len(all_images)))

In [20]:
# Define paths
PATHS = {
        "game": "./inputGTA/",
        "real": "./10k/all/",
        "degraded": "./Degraded/",
        "Temp_Game": "./Temp_Game/",
        "Temp_Real": "./Temp_Real/"
    }

In [21]:
def test_combination(combination, num_images=1000):
    """Test a specific combination of degradations and return metrics"""
    # Create temporary folder for degraded images
    temp_dir = "./temp_degraded/"
    if os.path.exists(temp_dir):
        shutil.rmtree(temp_dir)
    os.makedirs(temp_dir, exist_ok=True)
    
    # Get the pipeline function for this combination
    pipeline_func = generate_pipeline_combination(combination)
    
    # Select random real images to degrade
    real_images = select_random_images(PATHS["real"], num_images)
    
    # Apply degradation pipeline
    for filename in tqdm(real_images, desc=f"Processing {[f['name'] for f in combination]}"):
        try:
            img_path = os.path.join(PATHS["real"], filename)
            image = Image.open(img_path).convert('RGB')
            degraded = pipeline_func(image)
            degraded.save(os.path.join(temp_dir, filename))
        except Exception as e:
            print(f"Error processing {filename}: {e}")
    
    # Calculate metrics
    fid_score = fid.compute_fid(PATHS["Temp_Game"], temp_dir, mode="clean", model_name="clip_vit_b_32")
    kid_score = fid.compute_kid(PATHS["Temp_Game"], temp_dir, mode="clean")
    
    # Clean up
    shutil.rmtree(temp_dir)
    
    return {
        'combination': [f['name'] for f in combination],
        'fid': fid_score,
        'kid': kid_score
    }

In [22]:
def test_all_combinations(max_combo_size=3, num_images=100):
    """Test all possible combinations up to a certain size"""
    results = []
    total_combinations = sum(len(list(combinations(DEGRADATION_FUNCTIONS, i))) for i in range(1, max_combo_size+1))
    
    with tqdm(total=total_combinations, desc="Testing combinations") as pbar:
        # Generate all combinations from size 1 up to max_combo_size
        for combo_size in range(1, max_combo_size + 1):
            # Generate all possible combinations of this size
            for combo in combinations(DEGRADATION_FUNCTIONS, combo_size):
                # Test this combination
                result = test_combination(combo, num_images)
                results.append(result)
                pbar.update(1)
                
                # Print progress
                print(f"\nTested: {result['combination']}")
                print(f"FID: {result['fid']:.2f}, KID: {result['kid']:.4f}")
                print("-" * 50)
    
    # Convert results to DataFrame and save
    df = pd.DataFrame(results)
    df.to_csv("degradation_combination_results.csv", index=False)
    
    return df

In [23]:
def analyze_results(results_file="degradation_combination_results.csv"):
    """Analyze the results from combinatorial testing"""
    if not os.path.exists(results_file):
        print("No results file found. Run test_all_combinations() first.")
        return
    
    df = pd.read_csv(results_file)
    
    # Convert string representation of list to actual list
    df['combination'] = df['combination'].apply(eval)
    
    # Find best combinations (lower FID/KID is better)
    best_fid = df.loc[df['fid'].idxmin()]
    best_kid = df.loc[df['kid'].idxmin()]
    
    print("\n=== Best FID Combination ===")
    print(f"Functions: {best_fid['combination']}")
    print(f"FID: {best_fid['fid']:.2f}, KID: {best_kid['kid']:.4f}")
    
    print("\n=== Best KID Combination ===")
    print(f"Functions: {best_kid['combination']}")
    print(f"FID: {best_kid['fid']:.2f}, KID: {best_kid['kid']:.4f}")
    
    # Also look for good trade-offs between FID and KID
    df['combined_score'] = df['fid'] * 0.5 + df['kid'] * 0.5  # Adjust weights as needed
    best_combined = df.loc[df['combined_score'].idxmin()]
    
    print("\n=== Best Combined Score ===")
    print(f"Functions: {best_combined['combination']}")
    print(f"FID: {best_combined['fid']:.2f}, KID: {best_combined['kid']:.4f}")
    
    return df

In [46]:
results_df = test_all_combinations(max_combo_size=3, num_images=1000)
analyze_results()

Processing ['downscale_upscale']: 100%|██████████| 1000/1000 [00:17<00:00, 57.51it/s]


compute FID between two folders
Found 1000 images in the folder ./Temp_Game/


FID  : 100%|██████████| 32/32 [00:14<00:00,  2.16it/s]


Found 1000 images in the folder ./temp_degraded/


FID  : 100%|██████████| 32/32 [00:07<00:00,  4.12it/s]


...

KID  : 100%|██████████| 32/32 [00:21<00:00,  1.50it/s]


Found 1000 images in the folder ./temp_degraded/


Testing combinations: 100%|██████████| 92/92 [2:25:15<00:00, 94.73s/it] 


Tested: ['color_quantization', 'motion_blur', 'chromatic_aberration']
FID: 48.31, KID: 0.0484
--------------------------------------------------

=== Best FID Combination ===
Functions: ['median_filter', 'color_quantization', 'chromatic_aberration']
FID: 47.88, KID: 0.0417

=== Best KID Combination ===
Functions: ['enhance_edges']
FID: 59.69, KID: 0.0417

=== Best Combined Score ===
Functions: ['median_filter', 'color_quantization', 'chromatic_aberration']
FID: 47.88, KID: 0.0512


,combination,fid,kid,combined_score
0,[downscale_upscale],60.192449,0.043750,30.118099
1,[enhance_edges],59.693549,0.041729,29.867639
2,[gaussian_blur],58.987312,0.046121,29.516716
3,[median_filter],58.911805,0.045174,29.478490
4,[bilateral_filter],59.234567,0.048912,29.641740
...,...,...,...,...
87,"[median_filter, motion_blur, chromatic_aberrat...",54.269609,0.052645,27.161127
88,"[bilateral_filter, color_quantization, motion_...",54.587949,0.046311,27.317130
89,"[bilateral_filter, color_quantization, chromat...",48.470759,0.057833,24.264296
90,"[bilateral_filter, motion_blur, chromatic_aber...",55.592787,0.057954,27.825370


In [24]:
def test_promising_quadruplets(num_images=100):
    """Test targeted quadruplets based on best-performing triplet"""
    # Base triplet (your best performer)
    base = ['median_filter', 'color_quantization', 'chromatic_aberration']
    
    # Get additional functions not in base
    additional_funcs = [
        f for f in DEGRADATION_FUNCTIONS 
        if f['name'] not in base and f['name'] != 'pipeline'  # Exclude pipeline if present
    ]
    
    results = []
    
    # Test each quadruplet variation
    for func in additional_funcs:
        combo = base + [func['name']]
        print(f"\nTesting combination: {combo}")
        
        # Convert names to function dictionaries
        combo_funcs = [f for f in DEGRADATION_FUNCTIONS if f['name'] in combo]
        
        # Test this specific combination
        result = test_combination(combo_funcs, num_images)
        result['combination'] = combo  # Store the exact combination
        results.append(result)
    
    # Save and analyze results
    df = pd.DataFrame(results)
    df.to_csv("quadruplet_results.csv", index=False)
    
    # Print summary
    print("\n=== Quadruplet Testing Complete ===")
    print(f"Best FID: {df['fid'].min():.2f}")
    print(f"Best KID: {df['kid'].min():.4f}")
    
    return df

In [49]:
quadruplet_results = test_promising_quadruplets(num_images=1000)
analyze_results("quadruplet_results.csv")


Testing combination: ['median_filter', 'color_quantization', 'chromatic_aberration', 'downscale_upscale']


Processing ['downscale_upscale', 'median_filter', 'color_quantization', 'chromatic_aberration']: 100%|██████████| 1000/1000 [00:57<00:00, 17.54it/s]


compute FID between two folders
Found 1000 images in the folder ./Temp_Game/


FID  : 100%|██████████| 32/32 [00:15<00:00,  2.09it/s]


Found 1000 images in the folder ./temp_degraded/


...

KID  : 100%|██████████| 32/32 [00:21<00:00,  1.50it/s]


Found 1000 images in the folder ./temp_degraded/


KID  : 100%|██████████| 32/32 [00:11<00:00,  2.68it/s]



=== Quadruplet Testing Complete ===
Best FID: 47.97
Best KID: 0.0528

=== Best FID Combination ===
Functions: ['median_filter', 'color_quantization', 'chromatic_aberration', 'bilateral_filter']
FID: 47.97, KID: 0.0528

=== Best KID Combination ===
Functions: ['median_filter', 'color_quantization', 'chromatic_aberration', 'motion_blur']
FID: 48.08, KID: 0.0528

=== Best Combined Score ===
Functions: ['median_filter', 'color_quantization', 'chromatic_aberration', 'bilateral_filter']
FID: 47.97, KID: 0.0571


,combination,fid,kid,combined_score
0,"[median_filter, color_quantization, chromatic_...",48.394634,0.053334,24.223984
1,"[median_filter, color_quantization, chromatic_...",48.915248,0.053877,24.484562
2,"[median_filter, color_quantization, chromatic_...",48.099972,0.053790,24.076881
3,"[median_filter, color_quantization, chromatic_...",47.973532,0.057067,24.015299
4,"[median_filter, color_quantization, chromatic_...",48.082187,0.052840,24.067514


In [25]:
def test_five_stage_combinations(num_images=50):
    """Test 5-stage combinations building on best 4-stage result"""
    # Base quadruplet (your best performer)
    base = [
        'median_filter',
        'color_quantization', 
        'chromatic_aberration',
        'bilateral_filter'
    ]
    
    # Additional functions to test as 5th stage
    candidates = [
        'downscale_upscale',
        'enhance_edges',
        'gaussian_blur',
        'motion_blur'
    ]
    
    results = []
    
    for fifth_stage in candidates:
        combo = base + [fifth_stage]
        print(f"\nTesting 5-stage: {combo}")
        
        # Convert names to function dicts
        combo_funcs = [f for f in DEGRADATION_FUNCTIONS if f['name'] in combo]
        
        # Test with reduced image count for speed
        result = test_combination(combo_funcs, num_images)
        result['combination'] = combo
        results.append(result)
    
    # Save and analyze
    df = pd.DataFrame(results)
    df.to_csv("five_stage_results.csv", index=False)
    
    best = df.loc[df['fid'].idxmin()]
    print(f"\nBest 5-stage: {best['combination']}")
    print(f"FID: {best['fid']:.2f}, KID: {best['kid']:.4f}")
    
    return df

In [51]:
five_stage_results = test_five_stage_combinations(num_images=1000)


Testing 5-stage: ['median_filter', 'color_quantization', 'chromatic_aberration', 'bilateral_filter', 'downscale_upscale']


Processing ['downscale_upscale', 'median_filter', 'bilateral_filter', 'color_quantization', 'chromatic_aberration']: 100%|██████████| 1000/1000 [01:02<00:00, 15.90it/s]


compute FID between two folders
Found 1000 images in the folder ./Temp_Game/


FID  : 100%|██████████| 32/32 [00:15<00:00,  2.11it/s]


Found 1000 images in the folder ./temp_degraded/


...

compute KID between two folders
Found 1000 images in the folder ./Temp_Game/


KID  : 100%|██████████| 32/32 [00:21<00:00,  1.50it/s]


Found 1000 images in the folder ./temp_degraded/


KID  : 100%|██████████| 32/32 [00:11<00:00,  2.70it/s]



Best 5-stage: ['median_filter', 'color_quantization', 'chromatic_aberration', 'bilateral_filter', 'motion_blur']
FID: 47.51, KID: 0.0597


In [26]:
def test_six_stage_combinations(num_images=50):
    """Test 6-stage combos building on best 5-stage result"""
    # Base 5-stage (your best performer)
    base = [
        'median_filter',
        'color_quantization',
        'chromatic_aberration',
        'bilateral_filter',
        'motion_blur'
    ]
    
    # Sixth operation candidates
    candidates = [f for f in DEGRADATION_FUNCTIONS 
                 if f['name'] not in base and f['name'] in ['downscale_upscale', 'enhance_edges', 'gaussian_blur']]
    
    results = []
    
    for sixth_func in candidates:
        combo = base + [sixth_func['name']]
        print(f"\nTesting 6-stage: {combo}")
        
        # Convert names to function dicts
        combo_funcs = [f for f in DEGRADATION_FUNCTIONS if f['name'] in combo]
        
        # Test with reduced images for speed
        result = test_combination(combo_funcs, num_images)
        result['combination'] = combo
        results.append(result)
    
    # Save and analyze
    df = pd.DataFrame(results)
    df.to_csv("six_stage_results.csv", index=False)
    
    best = df.loc[df['fid'].idxmin()]
    print(f"\nBest 6-stage: {best['combination']}")
    print(f"FID: {best['fid']:.2f}, KID: {best['kid']:.4f}")
    
    return df

# Uncomment to run:
# six_stage_results = test_six_stage_combinations(num_images=50)

In [53]:
six_stage_results = test_six_stage_combinations(num_images=1000)


Testing 6-stage: ['median_filter', 'color_quantization', 'chromatic_aberration', 'bilateral_filter', 'motion_blur', 'downscale_upscale']


Processing ['downscale_upscale', 'median_filter', 'bilateral_filter', 'color_quantization', 'motion_blur', 'chromatic_aberration']: 100%|██████████| 1000/1000 [01:03<00:00, 15.69it/s]


compute FID between two folders
Found 1000 images in the folder ./Temp_Game/


FID  : 100%|██████████| 32/32 [00:15<00:00,  2.09it/s]


Found 1000 images in the folder ./temp_degraded/


...

compute KID between two folders
Found 1000 images in the folder ./Temp_Game/


KID  : 100%|██████████| 32/32 [00:21<00:00,  1.50it/s]


Found 1000 images in the folder ./temp_degraded/


KID  : 100%|██████████| 32/32 [00:12<00:00,  2.66it/s]



Best 6-stage: ['median_filter', 'color_quantization', 'chromatic_aberration', 'bilateral_filter', 'motion_blur', 'gaussian_blur']
FID: 47.60, KID: 0.0578


In [27]:
def test_seven_stage_combinations(num_images=50):
    """Test 7-stage combos building on best 6-stage result"""
    # Base 6-stage (best performer)
    base = [
        'median_filter',
        'color_quantization',
        'chromatic_aberration',
        'bilateral_filter',
        'motion_blur',
        'gaussian_blur'
    ]
    
    # Seventh operation candidates (exclude redundancies)
    candidates = [
        f for f in DEGRADATION_FUNCTIONS 
        if f['name'] not in base 
        and f['name'] in ['downscale_upscale', 'enhance_edges']
    ]
    
    results = []
    for seventh_func in candidates:
        combo = base + [seventh_func['name']]
        print(f"\nTesting 7-stage: {combo}")
        
        # Convert names to function dicts
        combo_funcs = [f for f in DEGRADATION_FUNCTIONS if f['name'] in combo]
        
        # Test with reduced images for speed
        result = test_combination(combo_funcs, num_images)
        result['combination'] = combo
        results.append(result)
    
    # Save and analyze
    df = pd.DataFrame(results)
    df.to_csv("seven_stage_results.csv", index=False)
    
    best = df.loc[df['fid'].idxmin()]
    print(f"\nBest 7-stage: {best['combination']}")
    print(f"FID: {best['fid']:.2f}, KID: {best['kid']:.4f}")
    return df

# Uncomment to run:
# seven_stage_results = test_seven_stage_combinations(num_images=50)

In [55]:
seven_stage_results = test_seven_stage_combinations(num_images=1000)


Testing 7-stage: ['median_filter', 'color_quantization', 'chromatic_aberration', 'bilateral_filter', 'motion_blur', 'gaussian_blur', 'downscale_upscale']


Processing ['downscale_upscale', 'gaussian_blur', 'median_filter', 'bilateral_filter', 'color_quantization', 'motion_blur', 'chromatic_aberration']: 100%|██████████| 1000/1000 [01:03<00:00, 15.74it/s]


compute FID between two folders
Found 1000 images in the folder ./Temp_Game/


FID  : 100%|██████████| 32/32 [00:15<00:00,  2.11it/s]


Found 1000 images in the folder ./temp_degraded/


...

compute KID between two folders
Found 1000 images in the folder ./Temp_Game/


KID  : 100%|██████████| 32/32 [00:21<00:00,  1.50it/s]


Found 1000 images in the folder ./temp_degraded/


KID  : 100%|██████████| 32/32 [00:11<00:00,  2.69it/s]



Best 7-stage: ['median_filter', 'color_quantization', 'chromatic_aberration', 'bilateral_filter', 'motion_blur', 'gaussian_blur', 'downscale_upscale']
FID: 47.54, KID: 0.0583


In [42]:
import os
import shutil
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
import itertools
import torch

In [43]:
# Disable multiprocessing for KID to prevent errors
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
torch.backends.cudnn.deterministic = True

# Define the specific operations to permute
best_ops = ['median_filter', 'color_quantization', 'chromatic_aberration']

In [45]:
# Configuration
PARAM_RANGES = {
    'median_filter': {'kernel_size': [3, 5, 7]},
    'color_quantization': {'n_colors': [32, 64]},
    'chromatic_aberration': {'max_shift': (1.0, 2.0)}
}
NUM_IMAGES = 1000
BATCH_SIZE = 100  # Adjust based on GPU memory

In [46]:
def safe_metric_calc(ref_dir, eval_dir):
    """Handle metric calculation with multiprocessing workaround"""
    try:
        # Calculate FID
        fid_score = fid.compute_fid(ref_dir, eval_dir, num_workers=0)  # Disable multiprocessing
        
        # Calculate KID with single worker
        kid_score = fid.compute_kid(ref_dir, eval_dir, num_workers=0)
        
        return fid_score, kid_score
    except Exception as e:
        print(f"Metric calculation failed: {str(e)}")
        return float('nan'), float('nan')


In [47]:
def process_permutation(order, num_images):
    """Process one permutation with batch support"""
    temp_dir = f"./temp_permtest/{'_'.join(order)}/"
    os.makedirs(temp_dir, exist_ok=True)
    
    # Generate random parameters
    params = {
        op: {k: np.random.choice(v) if isinstance(v, list) 
             else np.random.uniform(v[0], v[1])
             for k, v in PARAM_RANGES[op].items()}
        for op in order
    }
    
    # Process in batches
    processed = 0
    while processed < num_images:
        current_batch = min(BATCH_SIZE, num_images - processed)
        batch_images = select_random_images(PATHS["real"], current_batch)
        
        for filename in batch_images:
            img = Image.open(os.path.join(PATHS["real"], filename)).convert('RGB')
            img_tensor = pil_to_tensor(img)
            
            for op in order:
                if op == 'median_filter':
                    img_tensor = apply_median_filter(img_tensor, **params[op])
                elif op == 'color_quantization':
                    img_tensor = apply_color_quantization(img_tensor, **params[op])
                elif op == 'chromatic_aberration':
                    img_tensor = add_chromatic_aberration(img_tensor, **params[op])
            
            tensor_to_pil(img_tensor).save(os.path.join(temp_dir, f"{processed}_{filename}"))
            processed += 1
    
    # Calculate metrics
    fid_score, kid_score = safe_metric_calc(PATHS["Temp_Game"], temp_dir)
    shutil.rmtree(temp_dir)
    
    return {
        'order': order,
        'fid': fid_score,
        'kid': kid_score,
        **{f'{op}_params': params[op] for op in order}
    }

In [48]:
def test_all_permutations():
    """Main test function with error handling"""
    permutations = list(itertools.permutations(best_ops))
    results = []
    
    print(f"Testing {len(permutations)} permutations with {NUM_IMAGES} images each")
    
    for order in tqdm(permutations, desc="Permutations"):
        try:
            results.append(process_permutation(order, NUM_IMAGES))
        except Exception as e:
            print(f"Failed {order}: {str(e)}")
            continue
    
    # Save and display results
    df = pd.DataFrame(results)
    df.to_csv("permutation_results_1000img.csv", index=False)
    
    print("\n=== Top Performers ===")
    print(df.sort_values('fid').head().to_string(index=False))
    
    return df


In [49]:
results = test_all_permutations()

Testing 6 permutations with 1000 images each


Permutations:   0%|          | 0/6 [00:00<?, ?it/s]

compute FID between two folders
Found 1000 images in the folder ./Temp_Game/


FID  : 100%|██████████| 32/32 [00:24<00:00,  1.31it/s]


Found 1100 images in the folder ./temp_permtest/median_filter_color_quantization_chromatic_aberration/


...

compute KID between two folders
Found 1000 images in the folder ./Temp_Game/


KID  : 100%|██████████| 32/32 [00:24<00:00,  1.31it/s]


Found 1000 images in the folder ./temp_permtest/chromatic_aberration_color_quantization_median_filter/


Permutations: 100%|██████████| 6/6 [13:39<00:00, 136.52s/it]


=== Top Performers ===
                                                    order       fid      kid median_filter_params color_quantization_params       chromatic_aberration_params
(chromatic_aberration, color_quantization, median_filter) 83.926870 0.045520   {'kernel_size': 5}          {'n_colors': 64} {'max_shift': 1.5486217544720735}
(median_filter, chromatic_aberration, color_quantization) 83.995777 0.045390   {'kernel_size': 3}          {'n_colors': 64} {'max_shift': 1.1296465641492346}
(chromatic_aberration, median_filter, color_quantization) 87.307080 0.047760   {'kernel_size': 5}          {'n_colors': 32}  {'max_shift': 1.478661617997006}
(color_quantization, chromatic_aberration, median_filter) 90.277830 0.054565   {'kernel_size': 5}          {'n_colors': 64} {'max_shift': 1.8619243153669514}
(median_filter, color_quantization, chromatic_aberration) 91.354394 0.054427   {'kernel_size': 5}          {'n_colors': 32}   {'max_shift': 1.41712858447901}
